## Passo 1 - Descrição dos dados


In [1]:
# Imports
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import os

In [2]:
if (os.environ.get('DB_TYPE') == None):
    from dotenv import load_dotenv
    from config.definitions import ROOT_DIR
    load_dotenv(os.path.join(ROOT_DIR, 'config', 'conf', '.env'))

In [3]:
ROOT_DIR

'C:\\Users\\Bteix\\Desktop\\ComunidadeDS\\DS_ao_DEV\\PUBLISHING\\dags'

In [4]:
# Database Credentials

db_type = os.environ.get("DB_TYPE")
user = os.environ.get('DB_USER')
pswd = os.environ.get('DB_PASS')
host = os.environ.get('DB_HOST')
db_name = os.environ.get('DB_NAME')
print(db_type)

postgresql


In [6]:
engine = create_engine(f'{db_type}+psycopg2://{user}:{pswd}@{host}/{db_name}', echo=False
)

In [8]:
conn = engine.connect()

In [16]:
query = "SELECT table_name FROM information_schema.tables\
        WHERE table_schema = 'public'"
exec = conn.execute(query)

for table in exec.fetchall():
    print(table)

('vitrine_hm',)
('vitrine_marcy',)


In [22]:
query_h = """SELECT * FROM vitrine_hm"""
query_m = """SELECT * FROM vitrine_marcy"""

In [23]:
df_raw_hm = pd.read_sql(query_h, con=conn)
df_raw_macys = pd.read_sql(query_m, con=conn)

In [26]:
df_raw_hm.head()

,product_id,style_id,color_id,product_type,product_name,color_name,fit,price,model_size,model_height,cotton,rayon,lyocell,polyester,spandex,elastomultiester,scrap_datetime
0,1024256001,1024256,1,men_jeans_slim,slim_jeans,black,slim_fit,24.99,33/32,180.0,0.99,0.0,0.0,0.65,0.01,0.0,2023-01-30 21:06:18
1,1024256002,1024256,2,men_jeans_slim,slim_jeans,light_denim_blue,slim_fit,24.99,None,NaN,0.99,0.0,0.0,0.65,0.01,0.0,2023-01-30 21:06:18
2,1024256003,1024256,3,men_jeans_slim,slim_jeans,light_denim_blue,slim_fit,24.99,33/32,180.0,0.99,0.0,0.0,0.65,0.01,0.0,2023-01-30 21:06:18
3,1024256004,1024256,4,men_jeans_slim,slim_jeans,denim_blue,slim_fit,24.99,None,NaN,0.99,0.0,0.0,0.65,0.01,0.0,2023-01-30 21:06:18
4,1024256005,1024256,5,men_jeans_slim,slim_jeans,dark_blue,slim_fit,24.99,31/32,183.0,0.99,0.0,0.0,0.65,0.01,0.0,2023-01-30 21:06:18


In [27]:
df_raw_macys.head()

,product_id,style_id,color_id,product_type,product_name,color_name,fit,price,size,composition,material_1,material_2,material_3,material_4,material_5,scrap_datetime
0,2834585,32808,6823802.0,slim,men_512_slim_taper_fit_jeans,charred_marshmallow,taper_fit,79.5,28/30,cotton/elastane,cotton,elastane,None,None,None,2023-01-30 20:32:01
1,2834585,32808,2383464.0,slim,men_512_slim_taper_fit_jeans,dark_hollow_waterless,taper_fit,79.5,28/30,cotton/elastane,cotton,elastane,None,None,None,2023-01-30 20:32:01
2,2834585,32808,5365049.0,slim,men_512_slim_taper_fit_jeans,goldenrod,taper_fit,79.5,28/30,cotton/elastane,cotton,elastane,None,None,None,2023-01-30 20:32:01
3,2834585,32808,5584522.0,slim,men_512_slim_taper_fit_jeans,native_cali,taper_fit,79.5,28/30,cotton/elastane,cotton,elastane,None,None,None,2023-01-30 20:32:01
4,2834585,32808,2713262.0,slim,men_512_slim_taper_fit_jeans,sin_city_waterless,taper_fit,79.5,28/30,cotton/elastane,cotton,elastane,None,None,None,2023-01-30 20:32:01


In [ ]:
df_1 = df_raw.copy()

### Dimension data

In [ ]:
print('Number of rows:', df_1.shape[0])
print('Number of columns:', df_1.shape[1])

### Data types


In [ ]:
df_1.dtypes

In [ ]:
df_1['scrappy_datetime'] = pd.to_datetime(df_1['scrappy_datetime'])

### Missing data

In [ ]:
df_1.isna().sum()

In [ ]:
df_1.isna().sum()/df_1.shape[0]

### Handling missing data

In [ ]:
df_aux = df_1.drop(columns=['model_size', 'model_height'])

### 1.5 Data Description


In [ ]:
df_num = df_1.select_dtypes(include=['float64', 'int64'])
cat_attributes = df_1.select_dtypes(exclude=['float64', 'int64','datetime64[ns]'])

##### 1.5.1 Numeric

In [ ]:
# Central tendency and Dispersion
summary  = df_num.describe(percentiles=[0.01,0.25,0.5,0.75,0.99]).T
summary['skew'] = df_num.skew()
summary['kurtosis'] = df_num.kurtosis()
summary



##### Mínimo
 - Menor valor do conjunto de dados

##### Máximo
- Mairo valor do conjunto de dados

##### Range
- Estimativa da variabilidade ou dispersão
- Def: A diferença entr o valor máximo e o valor mínimo

##### Média
 - Definição: A média é a soma de todos os valores do conjunto de dados dividido pelo número de valores do conjunto.
 - Sensível a valores muito grandes ou muito pequenos (outliers)


In [ ]:
# np.mean()

##### Mediana
 - Def: É o valor central de um conjunto de dados ordenados.
 - Tamanho par -> média dos valores centrais
 - Tamanho ímpar -> (n+1)/2 é a posição da mediana
 - É uma medida mais robusta do que a média (não sofre efeito de valores muito elevados ou muito baixos — outliers)

In [ ]:
# np.median()

##### Desvio Padrão
 - Def: Estimativa de variabilidade ou dispersão
 - Mostra o quão dispersos dos valores do conjunto estão em relação a média. (distância média dos valores em relação a média.)

In [ ]:
# np.std(ddof=1)

##### Skewness

 - Def: Medida do quanto uma distribuição de dados desvia de uma distribuição normal — perfeitamente simétrica. 

  - Skewness Positiva e Negativa
  
    - mean = median -> simetric -> skew = 0

    - median > mean -> skews left (negative -> mean - median = -)

    - median < mean ->  skews right (positive -> mean - median = +)

 - Mostra a concentração dos dados de um lado da média.
 - Algoritmos de Machine Learning precisam treinar com dados não enviesados, ou seja, a distribuição da variável target deve ser mais simétrica possível.

##### Kurtosis

 - Def: Descreve a distribuição dos dados em torno da média.
  - Evidencia uma distribuição de caudas longas (probabilidade baixa de acontecer valores extremos).
  Kurtosis baixa -> maior probabilidade de valores extremos ocorrerem. 

#### 1.5.2 Categorical

## Passo 2 - Feature Engineering

In [ ]:
df_aux

In [ ]:
df_aux[['product_id', 'product_name',  'color_name', 'price']].groupby([ 'product_name',  'color_name']).mean()

## Passo 3 - Filtragem de atributos

In [ ]:
# Passo 4 - EDA ( Exploratory Data Analysis)